In [ ]:
using ITensors: MPO, MPS, OpSum, expect, inner, siteinds 
using ITensors
using ITensorMPS
using Plots
using QuanticsTCI
import TensorCrossInterpolation as TCI
using Quantics
using HDF5 

include("MPO_SCF_hopping.jl") 
include("get_functions.jl")

In [ ]:
#Antiferro guess
function initial_guess_trivial_up(L)
    
    xvals =  range(0, (2^L - 1); length=2^L)
    f(x) =  ((x )%2)  
    qtt, ranks, errors = quanticscrossinterpolate(Float64, f,  xvals; tolerance=1e-8)
    tt = TCI.tensortrain(qtt.tci)
    density_mps = ITensors.MPS(tt;sites)
    density_mpo = outer(density_mps',density_mps) 
    
    for i in 1:L
        density_mpo.data[i] = Quantics._asdiagonal(density_mps.data[i],sites[i])
    end
    
    return qtt, density_mpo,density_mps
end

function initial_guess_trivial_down(L)
    
    xvals =  range(0, (2^L - 1); length=2^L)
    f(x) =  ((x+1)%2)  
    qtt, ranks, errors = quanticscrossinterpolate(Float64, f,  xvals; tolerance=1e-8)
    tt = TCI.tensortrain(qtt.tci)
    density_mps = ITensors.MPS(tt;sites)
    density_mpo = outer(density_mps',density_mps) 
    
    for i in 1:L
        density_mpo.data[i] = Quantics._asdiagonal(density_mps.data[i],sites[i])
    end
    
    return qtt, density_mpo,density_mps
end

In [ ]:
function MPO_hop(L)
    k_1 = 2 * pi / (5  * sqrt(2))
    k_2 = 8 * pi / (  2^29* sqrt(3))
    t_0 = 1
    t_1 = 0.2
    
    t_2 = 0.2
    
    xvals = range(1, (2^L); length=2^L)
    f(x) =  t_0  + t_2 * cos(k_2 * (2 * x + 1 - 2^30)/2) + t_1 * cos(k_1 * (2 * x + 1 - 2^30)/2)
    qtt, ranks, errors = quanticscrossinterpolate(Float64, f,  xvals; tolerance=1e-8)
    tt = TCI.tensortrain(qtt.tci)
    density_mps = ITensors.MPS(tt;sites)
    density_mpo = outer(density_mps',density_mps) 
    
    for i in 1:L
        density_mpo.data[i] = Quantics._asdiagonal(density_mps.data[i],sites[i])
    end
    
    return qtt, density_mpo
end

"""
#with domain wall
function MPO_hop(L)
    k_1 = 2 * pi / (5* sqrt(2))
    k_2 = 8 * pi / (  2^29* sqrt(3))
    t_0 = 1
    t_1 = 0.2
    
    t_2 = 0.2
    delta = 0.2
    W = 2^30/40
    xvals = range(1, (2^L); length=2^L)
    f(x) =  t_0  + t_2 * cos(k_2 *(1 + delta * tanh((2 * x + 1 - 2^30)/(2*W))) * (2 * x- 2^30 + 1)/2) + t_1 * cos(k_1 * (2 * x - 2^30 + 1)/2)
    qtt, ranks, errors = quanticscrossinterpolate(Float64, f,  xvals; tolerance=1e-8)
    tt = TCI.tensortrain(qtt.tci)
    density_mps = ITensors.MPS(tt;sites)
    density_mpo = outer(density_mps',density_mps) 
    
    for i in 1:L
        density_mpo.data[i] = Quantics._asdiagonal(density_mps.data[i],sites[i])
    end
    
    return qtt, density_mpo
end
"""

In [ ]:
#making all calculations with same siteinds, flexible to modify
f = h5open("template_mps.h5","r")
templ = read(f,"template_mps",MPS)
close(f)


ini_rand = templ;
sites = siteinds(ini_rand);

In [ ]:
L = 30
U = 2.7
N = 250

sites = siteinds("Qubit",L ,conserve_qns= false)

id = OpSum()
for j = 1:L
    id += 1,"Id",j
end

#not divided by L here, normalized in the SCF.jl file
Id_op = MPO(id,sites);

In [ ]:
qtt_den_old_up, initial_den_op_up,ini_den_mps_up = initial_guess_trivial_up(L);
qtt_den_old_down, initial_den_op_down,ini_den_mps_down = initial_guess_trivial_down(L);

#getting diagonal MPO for the hopping MPO
qtt, hop_mpo = MPO_hop(L);

k_mpo = kinetic(L,sites,hop_mpo);

k_mpo_t = ITensorMPS.truncate!(k_mpo;cutoff=1e-8);

In [ ]:
max_iter = 120
@time qtt_den_new_up,qtt_den_new_down, Tn_list_up,Tn_list_down, conv_errr = SCF_Hubbard(L,
sites,U,k_mpo_t,max_iter,N,1e-3,initial_den_op_up,initial_den_op_down,ini_den_mps_up,ini_den_mps_down,qtt_den_old_up,qtt_den_old_down,1);#the mix


get_error_plot(conv_errr) 

In [ ]:
#save tn for later calculation
folder_name = "tn_billion_250cheby_domain_U_down"
 
if !isdir(folder_name)
    mkdir(folder_name)
    println("folder '$folder_name' is created")
else
    println("folder '$folder_name' exists")
end
 
for (i, hampo) in enumerate( Tn_list_down)
 
    file_path = joinpath(folder_name, "myfilempo_$i.h5")
    
 
    h5open(file_path, "w") do f
        write(f, "hampo",hampo)
    end
 
end

In [ ]:
folder_name = "tn_billion_250cheby_U_up"
 
if !isdir(folder_name)
    mkdir(folder_name)
    println("folder '$folder_name' is created")
else
    println("folder '$folder_name' exists")
end
 
for (i, hampo) in enumerate( Tn_list_up)
 
    file_path = joinpath(folder_name, "myfilempo_$i.h5")
    
 
    h5open(file_path, "w") do f
        write(f, "hampo",hampo)
    end
 
end